# Data Preparation

## Import the data

In [18]:
import pandas as pd
saps_df = pd.read_csv("SAPS2016_SA2017.csv").fillna(0)
#saps_df

In [19]:
vac_df = pd.read_csv("Vacant_Housing_Data_2016.csv", encoding="latin_1").fillna(0)
#vac_df

## Join data frames

In [20]:
sav_df = pd.merge(saps_df.rename(columns={'GUID' : 'SA_2017_GUID'}), vac_df, on='SA_2017_GUID')
#sav_df

## Function for calculating the three averages

In [58]:
def calc_averages (df, startIndex, multipliers, dividers) :
    return df.iloc[:, startIndex:(startIndex + len(multipliers))].dot(multipliers) / dividers

## Create the 'average male age' and 'average female age' columns

In [61]:
import numpy as np
# array containing ages for age categories (0 - 19 and then the 5-year ranges up to over-85)
ages = np.concatenate([np.array(range(20)), np.array(range(14)) * 5 + 22]).astype('float')
# calculating the average ages
sav_df['avg_age_male'] = calc_averages(sav_df, 3, ages, sav_df.iloc[:, (3 + len(ages))])
sav_df['avg_age_female'] = calc_averages(sav_df, 4 + len(ages), ages, sav_df.iloc[:, (4 + 2 * len(ages))])
#sav_df


## Create the 'average commuting time' column

In [62]:
# array containing ages for age categories (0 - 19 and then the 5-year ranges up to over-85)
commute_times = np.array([7.5, 22.5, 37.5, 52.5, 75, 120])
# calculating the average commute time
startI = 691
sav_df['avg_commute_time'] = calc_averages(sav_df, startI, commute_times, sav_df[sav_df.columns[startI + len(commute_times) + 1]] - sav_df[sav_df.columns[startI + len(commute_times)]])
# sav_df

## Get proportion of dwellings that are flats or apartments

In [63]:
nums_flats_or_apts = sav_df['T6_1_FA_H'] + sav_df['T6_1_BS_H']
sav_df['prop_flats_or_apartments'] = nums_flats_or_apts / sav_df['T6_1_TH']
sav_df['prop_flats_or_apts_high'] = sav_df['prop_flats_or_apartments'] > 0.2
print(sav_df['prop_flats_or_apts_high'].value_counts())
# sav_df

False    15431
True      3210
Name: prop_flats_or_apts_high, dtype: int64


## Get the number of vacant dwellings that are for sale or for rent

In [64]:
sav_df['vac_sal_or_ren'] = sav_df['VT2_TOT_SAL'] + sav_df['VT2_TOT_REN']
sav_df['prop_sal_or_ren'] = sav_df['vac_sal_or_ren'] / sav_df ['T6_1_TH']

# Analysis

In [65]:
# import scipy
import scipy.stats as ss
# list the relevant columns
sav_df.columns[-7:]

Index(['avg_age_male', 'avg_age_female', 'avg_commute_time',
       'prop_flats_or_apartments', 'prop_flats_or_apts_high', 'vac_sal_or_ren',
       'prop_sal_or_ren'],
      dtype='object')

## Correlation between male average age and vacancy

### counts

In [66]:
r, p = ss.pearsonr(sav_df['avg_age_male'], sav_df['vac_sal_or_ren'])
print("r = " + str(r))
print("p = " + str(p))

r = -0.007702995936952586
p = 0.2929597539925381


### proportions

In [67]:
r, p = ss.pearsonr(sav_df['avg_age_male'], sav_df['prop_sal_or_ren'])
print("r = " + str(r))
print("p = " + str(p))

r = 0.02092199121777958
p = 0.0042814656088646826


## Correlation between female average age and vacancy

### counts

In [68]:
r, p = ss.pearsonr(sav_df['avg_age_female'], sav_df['vac_sal_or_ren'])
print("r = " + str(r))
print("p = " + str(p))

r = -0.01648850611647644
p = 0.024372141058710228


### proportions

In [69]:
r, p = ss.pearsonr(sav_df['avg_age_female'], sav_df['prop_sal_or_ren'])
print("r = " + str(r))
print("p = " + str(p))

r = 0.008096721807610915
p = 0.2689837080796398


## Correlation between average commute time and vacancy

### counts

In [70]:
r, p = ss.pearsonr(sav_df['avg_commute_time'], sav_df['vac_sal_or_ren'])
print("r = " + str(r))
print("p = " + str(p))

r = 0.0046053603930058815
p = 0.5295189898857963


### proportions

In [71]:
r, p = ss.pearsonr(sav_df['avg_commute_time'], sav_df['prop_sal_or_ren'])
print("r = " + str(r))
print("p = " + str(p))

r = -0.019206327055532067
p = 0.008732693591177695


## Correlation between proportion of dwellings that are flats or apartments and vacancy

### counts

In [72]:
r, p = ss.pearsonr(sav_df['prop_flats_or_apartments'], sav_df['vac_sal_or_ren'])
print("r = " + str(r))
print("p = " + str(p))

r = 0.2676496044904768
p = 3.076902697922047e-303


### proportions

In [73]:
r, p = ss.pearsonr(sav_df['prop_flats_or_apartments'], sav_df['prop_sal_or_ren'])
print("r = " + str(r))
print("p = " + str(p))

r = 0.25042638759365765
p = 1.708867401943614e-264


## Correlation between proportion of dwellings that are flats or apartments (as boolean for high/low) and vacancy

### counts

In [74]:
f, p = ss.f_oneway(sav_df['vac_sal_or_ren'][sav_df['prop_flats_or_apts_high']], sav_df['vac_sal_or_ren'][np.logical_not(sav_df['prop_flats_or_apts_high'])])
print("f-statistic: " + str(f) + "\np-value: " + str(p))

f-statistic: 1342.360232045086
p-value: 7.605494146345699e-284


### proportions

In [75]:
f, p = ss.f_oneway(sav_df['prop_sal_or_ren'][sav_df['prop_flats_or_apts_high']], sav_df['prop_sal_or_ren'][np.logical_not(sav_df['prop_flats_or_apts_high'])])
print("f-statistic: " + str(f) + "\np-value: " + str(p))

f-statistic: 1148.0260664825123
p-value: 2.949491879534679e-244


## Connection between vacancy and type of dwelling

In [76]:
# table preparation
apt_flat_bedsit_count = sav_df['T6_1_FA_H'].sum() + sav_df['T6_1_BS_H'].sum()
other_dwelling_count = sav_df['T6_1_TH'].sum() - apt_flat_bedsit_count
vacant_apt_flat_bedsit_count = sav_df['VT1_TOT_PUR'].sum() + sav_df['VT1_TOT_CON'].sum() + sav_df['VT1_TOT_BED'].sum()
vacant_other_dwelling_count = sav_df['VT1_TOT_TOT'].sum() - vacant_apt_flat_bedsit_count
occupied_apt_flat_bedsit_count = apt_flat_bedsit_count - vacant_apt_flat_bedsit_count
occupied_other_dwelling_count = other_dwelling_count - vacant_other_dwelling_count

apt_flat_vacant_ct = pd.DataFrame({'occupied': [occupied_apt_flat_bedsit_count, occupied_other_dwelling_count], 'vacant': [vacant_apt_flat_bedsit_count, vacant_other_dwelling_count]})
apt_flat_vacant_ct.index = ['flat_apt_bedsit', 'other_dwelling']
apt_flat_vacant_ct

,occupied,vacant
flat_apt_bedsit,160953.0,43192.0
other_dwelling,1358024.0,140120.0


In [77]:
# apply test
chisq, p, dof, evs = ss.chi2_contingency(apt_flat_vacant_ct)
print("chi-squared " + str(chisq))
print("p = " + str(p))
print("degrees of freedom = " + str(dof))
print("expected value table: ")
print(evs)

chi-squared 26053.49060834214
p = 0.0
degrees of freedom = 1
expected value table: 
[[ 182161.52466767   21983.47533233]
 [1336815.47533233  161328.52466767]]


## Linear regression

In [ ]:
import statsmodels.api as sm
lm_indep_sav_df = sav_df[['avg_age_male', 'avg_age_female', 'avg_commute_time', 'prop_flats_or_apartments']]
lm_dep_sav_seq = sav_df[['prop_sal_or_ren']]

sm.add_constant(lm_indep_sav_df)
model = sm.OLS(lm_dep_sav_seq, lm_indep_sav_df)
results = model.fit()
print(results.summary())